**Import pyspark.sql**

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tugas_9').getOrCreate()

24/10/23 01:33:27 WARN Utils: Your hostname, codespaces-d86cfb resolves to a loopback address: 127.0.0.1; using 10.0.3.190 instead (on interface eth0)
24/10/23 01:33:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/23 01:33:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/23 01:33:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 1. Basic SQL Operations in Spark SQL

In [11]:
data = [
    ('James', 34, 'M', 3000, 1),
    ('Anna', 28, 'F', 4100, 2),
    ('Lee', 23, 'M', 2700, 1)
]
columns = ['Name', 'Age', 'Gender', 'Salary', 'DeptId']
df = spark.createDataFrame(data, schema=columns)
df.createOrReplaceTempView('employees')
spark.sql('SELECT * FROM employees').show()
spark.sql('SELECT Name, Age FROM employees WHERE Salary > 3000').show()
spark.sql('SELECT AVG(Salary) FROM employees').show()


+-----+---+------+------+------+
| Name|Age|Gender|Salary|DeptId|
+-----+---+------+------+------+
|James| 34|     M|  3000|     1|
| Anna| 28|     F|  4100|     2|
|  Lee| 23|     M|  2700|     1|
+-----+---+------+------+------+

+----+---+
|Name|Age|
+----+---+
|Anna| 28|
+----+---+

+------------------+
|       avg(Salary)|
+------------------+
|3266.6666666666665|
+------------------+



### 2. Advanced Queries for Data Analysis

In [4]:
spark.sql('''
SELECT Gender, SUM(Salary) as TotalSalary, Age
FROM employees
GROUP BY Gender, Age
ORDER BY Age
''').show()

+------+-----------+---+
|Gender|TotalSalary|Age|
+------+-----------+---+
|     M|       2700| 23|
|     F|       4100| 28|
|     M|       3000| 34|
+------+-----------+---+



In [13]:
# 1. Cari rata-rata gaji per departemen.
spark.sql('SELECT * FROM employees').show()
print('lihat rata rata per departemen')
spark.sql('''
SELECT deptId, AVG(Salary) as avgSalary
FROM employees
GROUP BY deptId
''').show()

+-----+---+------+------+------+
| Name|Age|Gender|Salary|DeptId|
+-----+---+------+------+------+
|James| 34|     M|  3000|     1|
| Anna| 28|     F|  4100|     2|
|  Lee| 23|     M|  2700|     1|
+-----+---+------+------+------+

lihat rata rata per departemen
+------+---------+
|deptId|avgSalary|
+------+---------+
|     1|   2850.0|
|     2|   4100.0|
+------+---------+



In [29]:
# 2. Temukan karyawan yang memiliki gaji di atas rata-rata untuk gender masing-masing.
spark.sql('SELECT * FROM employees').show()
print('Karyawan yg gajinya diatas rata rata untuk tiap gender')

avgPerGender = spark.sql('''
SELECT Gender, AVG(Salary) as avgSalary
FROM employees
GROUP BY Gender
''')

spark.sql('''
SELECT e.Name, e.Gender, e.Salary
FROM employees e
JOIN (
    SELECT Gender, AVG(Salary) as avgSalary
    FROM employees
    GROUP BY Gender
) AS avgPerGender
ON e.Gender = avgPerGender.Gender
WHERE e.Salary > avgPerGender.avgSalary
''').show()




+-----+---+------+------+------+
| Name|Age|Gender|Salary|DeptId|
+-----+---+------+------+------+
|James| 34|     M|  3000|     1|
| Anna| 28|     F|  4100|     2|
|  Lee| 23|     M|  2700|     1|
+-----+---+------+------+------+

Karyawan yg gajinya diatas rata rata untuk tiap gender


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '=>'.(line 10, pos 15)

== SQL ==

SELECT e.Name, e.Gender, e.Salary
FROM employees e
JOIN (
    SELECT Gender, AVG(Salary) as avgSalary
    FROM employees
    GROUP BY Gender
) AS avgPerGender
ON e.Gender = avgPerGender.Gender
WHERE e.Salary => avgPerGender.avgSalary
---------------^^^
